# HTM - Preparación

## Carga de datos

Cargamos librerías y datos:

Las típicas (pandas, matplotlib, numpy)...

Funciones de sklearn de preprocesado y métricas.

Modelos y métricas de PyOD.

In [1]:
import pandas as pd
import numpy as np
from time import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import cohen_kappa_score

from pyod.models.mad import MAD

random_state = np.random.RandomState(42)

In [2]:
df = pd.read_csv('../../../..../../../Datasets/Dataset_2.csv',index_col='fecha')
df.index = pd.to_datetime(df.index)
df.head(2)

,FormacionNIRHumedadPV,FibraticPredNIRHumedadPV,Hum_Pred,Etapa2MWHumedadPV,ExtractorVelocidadPV,FormacionAlturaMantaPV,FormadoraVelocidadPV,FormadoraSiloNivel,SiloFibraNivel,SiloFibraVelocidadPV,...,ScalperReservaIzqPosPV,FormacionNIRPH,FormacionNIRHumedadPV_std,FibraticPredNIRHumedadPV_std,Hum_Pred_std,Etapa2MWHumedadPV_std,Negro,CurvaCola,Congelado,Hum
fecha,,,,,,,,,,,,,,,,,,,,,
2021-02-10 10:00:00,0.0,6.465569,10.920920,13.141570,50.55861,120.965,37.6,70.741330,12.590190,26.1,...,66.029085,0.0,0.0,0.022282,0.163959,0.221663,1,0,1,0
2021-02-10 10:01:00,0.0,6.355772,10.604865,12.412745,50.55929,112.285,29.6,72.317965,9.990133,21.4,...,68.508440,0.0,0.0,0.029155,0.060918,0.151328,1,0,1,0


Unimos todas las anomalías en una columna, para posteriormente realizar las métricas tanto para todas las anomalías como para cada una en concreto.

Para ello, sumo las columnas de anomalías, y después las que son mayores que cero las establezco como uno (porque significa que alguna de las columnas si tenía anomalía registrada), las demás como cero.

In [3]:
df['Anomalia'] = df['Negro'] + df['Congelado'] + df['Hum']
df['Anomalia'] = df['Anomalia'].map(lambda x: 1 if x!=0 else 0)

df = df.drop(df[df['Anomalia']==1].index)
df = df.drop(['Negro','Congelado','Hum','Anomalia'], axis=1)

lista_anomalias = ['CurvaCola']

atributos = df.columns.drop(lista_anomalias)
len(atributos)

22

In [4]:
df.to_csv('Datos/Entrada_limpia.csv')

In [5]:
#df = df['FormacionNIRHumedadPV']
df = df.iloc[40000:60000,]
df = df.iloc[:,0:1]

In [6]:
df

,FormacionNIRHumedadPV
fecha,
2021-03-24 05:22:00,8.65
2021-03-24 05:23:00,8.71
2021-03-24 05:24:00,8.77
2021-03-24 05:25:00,8.86
2021-03-24 05:26:00,8.96
...,...
2021-04-15 08:19:00,8.27
2021-04-15 08:20:00,7.79
2021-04-15 08:21:00,7.23


In [7]:
df.to_csv('Datos/Entrada.csv', header=None)

In [8]:
filename = "Datos/Entrada.csv"

aux = open(filename,"r")
originaltext = aux.read()
aux.close()

aux = open(filename,"w")
aux.write("fecha,FormacionNIRHumedadPV \ndatetime,float \nT, \n")
aux.write(originaltext)
aux.close()